In [3]:
#本代码作者 宋鹏飞 西安交通大学 数学与统计学院
#时间2020年03月08日
#本代码为突发疾病多阶段多尺度模型核心算法， 以武汉市和北京市为例
#由于作者没有受过系统的编程训练，纯属业余，代码过于粗糙，有编程‘洁癖’的敬请谅解
import pandas as pd
import numpy as np
import time
start=time.time() #计算模拟时间 
#初始时间1月9日24时
Tspan=53 #初始时刻, 可修改
dt=1/3 #单位时间, 可修改
Tlock=14 #封城时间 1月23日封城

Tbeijinghome=15 #1月24日一级响应
Tbeijingrelease=32  #逐渐恢复市内交通
Twuhanhome=14  #武汉home
Thubeirelease=114 #湖北五月初恢复交通

Trelease=145 #湖北六月初离汉通道开启
#城市属性  可自行修改城市及区
Citys=['Beijing','Wuhan']
Wuhanqu=['江岸区','江汉区','硚口区','汉阳区','武昌区','青山区',
         '洪山区','蔡甸区','江夏区','黄陂区','新洲区','东西湖区','汉南区']
Beijingqu=['密云区','延庆区','朝阳区','丰台区','石景山区','海淀区','门头沟区', 
           '房山区','通州区','顺义区','昌平区','大兴区','怀柔区','平谷区','东城区','西城区']

Wuhan_renkou=10000*np.array([96.24,72.96,86.85,65.27,127.63,52.88,163.75,
                       73.50,91.37,98.83,90.21,56.25,13.55]) #武汉人口

WUHAN=pd.DataFrame(data=Wuhan_renkou,index=Wuhanqu,columns=['Renkou'])
Beijing_renkou=10000*np.array([49.5,34.8,360.5,210.5,59.0,335.8,33.1,118.8,157.8,116.9,210.8,179.6,
                        41.4,45.6,82.2,117.9]) #北京人口
BEIJING=pd.DataFrame(data=Beijing_renkou,index=Beijingqu,columns=['Renkou'])
Patches={'Beijing':Beijingqu,'Wuhan':Wuhanqu}

#Wuhancases=np.array([258,363,425,495,572,618,698,1590,1905,2261,2639,3215,4109,5142,6384,8351,10117,11618,13603,14982,16902,18454,19558,...
   #20630+12364,21960+14031,37914,39462,41152,42752,44412,45027,45346,45660])

Atr=pd.DataFrame(index=Citys)#城市属性
Atr['Pnum']=np.zeros(len(Citys))
for i in Citys:#添加城市属性区的数目
    Atr.loc[i,'Pnum']=len(Patches[i])
Atr['Province']=['Beijing','Hubei'] #添加城市省份


# 按照模型将人群分为c*w*h个子类，c所在城市, w工作所在地， h为家庭所在地
INDEX=[]
for c in Citys:
    for w in Patches[c]:
        for h in Patches[c]:
            INDEX.append(c+h+w)
            
            
# 输入的每个人群子类的参数属性
PARA=pd.DataFrame(index=INDEX,columns=['Cityname','S_initial','E_initial',
                           'I_initial','R_initial','D_initial','H_initial'])
for c in Citys:#更新城市名字
    for i in PARA.index:
        if c in i:
            PARA.loc[i,'Cityname']=c
PARA['S_initial']=np.ones(len(INDEX))
c='Wuhan'
for h in Patches[c]:
    for w in Patches[c]:
        if h==w:
            PARA.loc[c+h+w,'S_initial']=WUHAN.loc[h,'Renkou']/3+WUHAN.loc[h,'Renkou']**2/sum(Wuhan_renkou)*2/3
        else:
            PARA.loc[c+h+w,'S_initial']=2/3*WUHAN.loc[h,'Renkou']*WUHAN.loc[w,'Renkou']/sum(Wuhan_renkou)
c='Beijing'
for h in Patches[c]:
    for w in Patches[c]:
        if h==w:
            PARA.loc[c+h+w,'S_initial']=BEIJING.loc[h,'Renkou']/3+BEIJING.loc[h,'Renkou']**2/sum(Beijing_renkou)*2/3
        else:
            PARA.loc[c+h+w,'S_initial']=2/3*BEIJING.loc[h,'Renkou']*BEIJING.loc[w,'Renkou']/sum(Beijing_renkou)    
'''AAA=0
c='Wuhan'
for h in Patches[c]:
    for w in Patches[c]:
        AAA=AAA+PARA.loc[c+h+w,'S_initial']
print(AAA)'''
PARA['E_initial']=np.zeros(len(INDEX))
PARA.loc['Wuhan江汉区江汉区','E_initial']=150
PARA['I_initial']=np.zeros(len(INDEX))
PARA.loc['Wuhan江汉区江汉区','I_initial']=220
PARA['R_initial']=np.zeros(len(INDEX))
PARA['D_initial']=np.zeros(len(INDEX))
PARA.loc['Wuhan江汉区江汉区','D_initial']=1
PARA['H_initial']=np.zeros(len(INDEX))
PARA.loc['Wuhan江汉区江汉区','H_initial']=41
#print(PARA)


# 输出
#各个地方的S， E, I, R^{c,w,h}
COLUMNS=[] #每列
for i in INDEX:
    A=[i+'_S',i+'_E',i+'_I',i+'_R',i+'_D',i+'_H']
    COLUMNS.extend(A)
#print(COLUMNS)
Solution=pd.DataFrame(columns=COLUMNS)


#定义控制参数C
def Control(c,i,t): #city, location, time, initial time is Jan 9th,24:00 OK
    cnh=20#控制措施产生效果
    cnhk=0.14#效果
    cnhday=5#花了多少天
    cwh=20#控制措施产生效果
    cwhday=8#花了多少天
    cwhk=0.083#效果
    def cNH(t):#湖北以外控制
        if t<=cnh: #1月30 号之后
            c=1
        elif t>cnh+cnhday:
            c=cnhk  #可调
        else:
            c=np.exp(np.log(cnhk)*(t-cnh)/cnhday)
        return c
    def cH(t):#湖北控制
        if t<=cwh: #2月3号之后
            c=1
        elif t>cwh+cwhday:
            c=cwhk  #可调
        else:
            c=np.exp(np.log(cwhk)*(t-cwh)/cwhday)
        return c
    if Atr.loc[c,'Province']=='Hubei':
        control=cH(t)
    else:
        control=cNH(t)
    return control
#print(Control('Wuhan',16,26))
#定义潜伏感染者感染率
def Betae(c,i,t):#Exposed 感染率， i表示location, t时间 OK
    k=0.95
    if Atr.loc[c,'Province']=='Hubei':
        betae=0.24*k
    else:
        betae=0.20*k
    return betae
#print(Betae('Beijing',10))
#定义有症状感染者感染率
def Betai(c,i,t):#Infected 感染率， i表示location, t时间 OK
    k=0.95
    if Atr.loc[c,'Province']=='Hubei':
        betai=0.24*k
    else:
        betai=0.20*k
    return betai
#定义市内出行概率
def Prob(c,w,h,i,t): # c city, w work place, h, home place, i, location, t time OK
    if c=='Beijing':
        if t<=Tbeijinghome:   
            prob=1/Atr.loc[c,'Pnum']
        elif t>Tbeijinghome and t<Tbeijingrelease:
            if i==h:
                prob=1
            else:
                prob=0
        else:
            if (np.floor(t)-Tbeijingrelease+1)%7==5 or (np.floor(t)-Tbeijingrelease+1)%7==6:#双休日
                if t-np.floor(t)>0 and t-np.floor(t)<=1/3+0.001:#白天
                    prob=1/Atr.loc[c,'Pnum']
                else:
                    if i==h:
                        prob=1
                    else:
                        prob=0
            else:
                if t-np.floor(t)>0 and t-np.floor(t)<=1/3+0.001:#白天
                    if i==w:
                        prob=1
                    else:
                        prob=0
                else:
                    if i==h:
                        prob=1
                    else:
                        prob=0
    if c=='Wuhan':
        if t<=Twuhanhome:   
            prob=1/Atr.loc[c,'Pnum']
        elif t>Twuhanhome and t<Thubeirelease:
            if i==h:
                prob=1
            else:
                prob=0
        else:
            if (np.floor(t)-Thubeirelease+1)%7==5 or (np.floor(t)-Thubeirelease+1)%7==6:#双休日
                if t-np.floor(t)>0 and t-np.floor(t)<=1/3+0.001:#白天
                    prob=1/Atr.loc[c,'Pnum']
                else:
                    if i==h:
                        prob=1
                    else:
                        prob=0
            else:#工作日
                if t-np.floor(t)>0 and t-np.floor(t)<=1/3+0.001:#白天
                    if i==w:
                        prob=1
                    else:
                        prob=0
                else:
                    if i==h:
                        prob=1
                    else:
                        prob=0            
    return prob
'''for i in range(180):
    print(Prob('Wuhan','汉南区','江汉区','汉南区',i/3))'''
#定义城市与城市之间迁徙
def Diffusion(a,b,t): # a city to b city, distance induced diffusion rate OK
    if t>Tlock and t<Trelease:
        diffusion=0
    else:
        diffusion=0.00118
    return diffusion
def Migration(a,b,t):#a city to b city,migration probability OK
    if a=='Wuhan' and b=='Beijing':
        migration=1
    else:
        migration=0
    return migration
#定义迁入人群概率Q，减模型
def Qprob(c,w,h,t):# c city, w work place, h, home place, t time OK
    qprob=1/Atr.loc[c,'Pnum']**2
    return qprob
#定义恢复率
def Gamma(c,i,t): # c city, i patch, t time OK
    y=1/12.7
    return y
#定义因病死亡率
def Alpha(c,i,t): # c city, i patch, t time OK
    a=0;
    return a
#定义潜伏期长度
def Sigma(): #OK
    s=1/5.2
    return s

#定义中间变量
COLUMNS1=[] #每列
for i in INDEX:
    A=[i+'_S1',i+'_E1',i+'_I1',i+'_R1',i+'_D1',i+'_H1']
    COLUMNS1.extend(A)
COLUMNS2=[] #每列
for i in INDEX:
    A=[i+'_S2',i+'_E2',i+'_I2',i+'_R2',i+'_D2',i+'_H2']
    COLUMNS2.extend(A)
#print(COLUMNS1)
Midpar1=pd.DataFrame(data=np.zeros((1,len(COLUMNS1))),columns=COLUMNS1)
Midpar2=pd.DataFrame(data=np.zeros((1,len(COLUMNS2))),columns=COLUMNS2)

#print(type(Midpar2.iloc[:,0]))
#代入初值
#!!!!!!!算法开始，微分方程采取最简单的差分，可自行修改微分方程解法
for i in INDEX:
    for j in ['S','E','I','R','D','H']:
        Midpar1[i+'_'+j+'1']=PARA.loc[i,j+'_initial']
#print(Midpar1)
for i in range(int(Tspan/dt)):
    print(i)
    for c in Citys:
        #计算S_{i}^{c} E_{i}^{c} I R_{i}^{c}
        Nlocation=pd.DataFrame(data=np.zeros((len(Patches[c]),5)),
                          columns=['Ssum','Esum','Isum','Rsum','Nsum'],index=Patches[c])
        #print(type(Nlocation.loc[patch,'Ssum']))
        for patch in Patches[c]:
            for hh in Patches[c]:
                for ww in Patches[c]:
                    Suma=Nlocation.loc[patch,'Ssum']+Prob(c,ww,hh,patch,i*dt)*Midpar1.loc[0,c+hh+ww+'_S1']
                    #print(Suma)
                    #print(type(Prob(c,ww,hh,patch,i*dt)*Midpar1.loc[0,c+hh+ww+'_S1']))
                    Sumb=Nlocation.loc[patch,'Esum']+Prob(c,ww,hh,patch,i*dt)*Midpar1.loc[0,c+hh+ww+'_E1']
                    Sumc=Nlocation.loc[patch,'Isum']+Prob(c,ww,hh,patch,i*dt)*Midpar1.loc[0,c+hh+ww+'_I1']
                    Sumd=Nlocation.loc[patch,'Rsum']+Prob(c,ww,hh,patch,i*dt)*Midpar1.loc[0,c+hh+ww+'_R1']
                    Nlocation.loc[patch,'Ssum']=Suma
                    Nlocation.loc[patch,'Esum']=Sumb
                    Nlocation.loc[patch,'Isum']=Sumc
                    Nlocation.loc[patch,'Rsum']=Sumd
                    Nlocation.loc[patch,'Nsum']=Suma+Sumb+Sumc+Sumd
        Ntotal=sum(Nlocation['Nsum'].values)
        #print(Ntotal) OK
        #计算扩散项:迁出 和迁入 
        imigration_rate=0
        emigration_num_S=0
        emigration_num_E=0
        emigration_num_I=0
        emigration_num_R=0
        for b in Citys:
            imigration_rate=imigration_rate+Migration(c,b,i*dt)*Diffusion(c,b,i*dt)
            #计算 S^{b}
            SSUM=0
            ESUM=0
            ISUM=0
            RSUM=0
            for hb in Patches[b]:
                for wb in Patches[b]:
                    SSUM=SSUM+Midpar1.loc[0,b+hb+wb+'_S1']
                    #print(SSUM)
                    ESUM=ESUM+Midpar1.loc[0,b+hb+wb+'_E1']
                    ISUM=ISUM+Midpar1.loc[0,b+hb+wb+'_I1']
                    RSUM=RSUM+Midpar1.loc[0,b+hb+wb+'_R1']
            emigration_num_S=emigration_num_S+Migration(b,c,i*dt)*Diffusion(b,c,i*dt)*SSUM
            emigration_num_E=emigration_num_E+Migration(b,c,i*dt)*Diffusion(b,c,i*dt)*ESUM
            emigration_num_I=emigration_num_I+Migration(b,c,i*dt)*Diffusion(b,c,i*dt)*ISUM
            emigration_num_R=emigration_num_R+Migration(b,c,i*dt)*Diffusion(b,c,i*dt)*RSUM
        for h in Patches[c]:
            for w in Patches[c]:       
                Infection=0
                Recover=0
                Dead=0
                for patch in Patches[c]:
                    I_sum=Nlocation.loc[patch,'Isum']
                    E_sum=Nlocation.loc[patch,'Esum']
                    N_sum=Nlocation.loc[patch,'Nsum']
                    Infection=Infection+Prob(c,w,h,patch,i*dt)*Control(c,patch,i*dt)*(Betai(c,patch,i*dt)*I_sum/N_sum+Betae(c,patch,i*dt)*E_sum/N_sum)
                    Recover=Recover+Prob(c,w,h,patch,i*dt)*Gamma(c,patch,i*dt)
                    Dead=Dead+Prob(c,w,h,patch,i*dt)*Alpha(c,patch,i*dt)
                #print(Recover) OK
                qprob=(Midpar1.loc[0,c+h+w+'_S1']+Midpar1.loc[0,c+h+w+'_E1']+Midpar1.loc[0,c+h+w+'_I1']+Midpar1.loc[0,c+h+w+'_R1'])/Ntotal
                Midpar2.loc[0,c+h+w+'_S2']=Midpar1.loc[0,c+h+w+'_S1']+dt*(-Midpar1.loc[0,c+h+w+'_S1']*Infection-imigration_rate*Midpar1.loc[0,c+h+w+'_S1']+qprob*emigration_num_S)
                Midpar2.loc[0,c+h+w+'_E2']=Midpar1.loc[0,c+h+w+'_E1']+dt*(Midpar1.loc[0,c+h+w+'_S1']*Infection-Sigma()*Midpar1.loc[0,c+h+w+'_E1']-imigration_rate*Midpar1.loc[0,c+h+w+'_E1']+qprob*emigration_num_E)
                Midpar2.loc[0,c+h+w+'_I2']=Midpar1.loc[0,c+h+w+'_I1']+dt*(Sigma()*Midpar1.loc[0,c+h+w+'_E1']-(Recover+Dead)*Midpar1.loc[0,c+h+w+'_I1']-imigration_rate*Midpar1.loc[0,c+h+w+'_I1']+qprob*emigration_num_I)
                Midpar2.loc[0,c+h+w+'_R2']=Midpar1.loc[0,c+h+w+'_R1']+dt*(Recover*Midpar1.loc[0,c+h+w+'_I1']-imigration_rate*Midpar1.loc[0,c+h+w+'_R1']+qprob*emigration_num_R)
                Midpar2.loc[0,c+h+w+'_D2']=Midpar1.loc[0,c+h+w+'_D1']+dt*Dead*Midpar1.loc[0,c+h+w+'_I1']
                Midpar2.loc[0,c+h+w+'_H2']=Midpar1.loc[0,c+h+w+'_H1']+dt*Sigma()*Midpar1.loc[0,c+h+w+'_E1']
                Midpar1.loc[0,c+h+w+'_S1']=Midpar2.loc[0,c+h+w+'_S2']
                Midpar1.loc[0,c+h+w+'_E1']=Midpar2.loc[0,c+h+w+'_E2']
                Midpar1.loc[0,c+h+w+'_I1']=Midpar2.loc[0,c+h+w+'_I2']
                Midpar1.loc[0,c+h+w+'_R1']=Midpar2.loc[0,c+h+w+'_R2']
                Midpar1.loc[0,c+h+w+'_D1']=Midpar2.loc[0,c+h+w+'_D2']
                Midpar1.loc[0,c+h+w+'_H1']=Midpar2.loc[0,c+h+w+'_H2']
                Solution.loc[i,c+h+w+'_S']=Midpar2.loc[0,c+h+w+'_S2']
                Solution.loc[i,c+h+w+'_E']=Midpar2.loc[0,c+h+w+'_E2']
                Solution.loc[i,c+h+w+'_I']=Midpar2.loc[0,c+h+w+'_I2']
                Solution.loc[i,c+h+w+'_R']=Midpar2.loc[0,c+h+w+'_R2']
                Solution.loc[i,c+h+w+'_D']=Midpar2.loc[0,c+h+w+'_D2']
                Solution.loc[i,c+h+w+'_H']=Midpar2.loc[0,c+h+w+'_H2']
#输出数据
#S^{c,h}
#每个城市各区感染人口
INDEX_Distinct=[]
for  c in Citys:
    for h in Patches[c]:
        INDEX_Distinct.append(c+h)
COLUMNS_Distinct=[] #每列
for i in INDEX_Distinct:
    A=[i+'_S',i+'_E',i+'_I',i+'_R',i+'_D',i+'_H']
    COLUMNS_Distinct.extend(A)
Solution_Distinct=pd.DataFrame(columns=COLUMNS_Distinct)
for i in range(int(Tspan/dt)):
    for c in Citys:
        for h in Patches[c]:
            S_ch=0
            E_ch=0
            I_ch=0
            R_ch=0
            D_ch=0
            H_ch=0
            for w in Patches[c]:
                S_ch=S_ch+Solution.loc[i,c+h+w+'_S']
                E_ch=E_ch+Solution.loc[i,c+h+w+'_E']
                I_ch=I_ch+Solution.loc[i,c+h+w+'_I']
                R_ch=R_ch+Solution.loc[i,c+h+w+'_R']
                D_ch=D_ch+Solution.loc[i,c+h+w+'_D']
                H_ch=H_ch+Solution.loc[i,c+h+w+'_H']
            Solution_Distinct.loc[i,c+h+'_S']=S_ch
            Solution_Distinct.loc[i,c+h+'_E']=E_ch
            Solution_Distinct.loc[i,c+h+'_I']=I_ch
            Solution_Distinct.loc[i,c+h+'_R']=R_ch
            Solution_Distinct.loc[i,c+h+'_D']=D_ch
            Solution_Distinct.loc[i,c+h+'_H']=H_ch
#各市感染人口

COLUMNS_City=[] #每列
for i in Citys:
    A=[i+'_S',i+'_E',i+'_I',i+'_R',i+'_D',i+'_H']
    COLUMNS_City.extend(A)
Solution_City=pd.DataFrame(columns=COLUMNS_City)           
for i in range(int(Tspan/dt)):
    for c in Citys:
        S_c=0
        E_c=0
        I_c=0
        R_c=0
        D_c=0
        H_c=0        
        for h in Patches[c]:
            S_c=S_c+Solution_Distinct.loc[i,c+h+'_S']
            E_c=E_c+Solution_Distinct.loc[i,c+h+'_E']
            I_c=I_c+Solution_Distinct.loc[i,c+h+'_I']
            R_c=R_c+Solution_Distinct.loc[i,c+h+'_R']
            D_c=D_c+Solution_Distinct.loc[i,c+h+'_D']
            H_c=H_c+Solution_Distinct.loc[i,c+h+'_H']
        Solution_City.loc[i,c+'_S']=S_c
        Solution_City.loc[i,c+'_E']=E_c
        Solution_City.loc[i,c+'_I']=I_c
        Solution_City.loc[i,c+'_R']=R_c
        Solution_City.loc[i,c+'_D']=D_c
        Solution_City.loc[i,c+'_H']=H_c
            
print(Solution_City)
end=time.time()
print('Running time: %s Seconds'%(end-start))

#保存数据
Solution_City.to_csv('Down95City202003302.csv',index=0) #市
Solution_Distinct.to_csv('Down95Distinct202003302.csv',index=0)#区
Solution.to_csv('Down95Detail202003302.csv',index=0)#人群子类


1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
       Beijing_S Beijing_E  Beijing_I   Beijing_R Beijing_D   Beijing_H  \
0    2.15463e+07     0.059  0.0865333           0         0           0   
1    2.15506e+07   0.13069   0.176054  0.00454243         0  0.00378205   
2    2.15548e+07  0.215445   0.269724   0.0137436         0   0.0121596   
3    2.15591e+07  0.313748   0.368697   0.0277614         0   0.0259702   
4    2.15634e+07   0.42619   0.474134   0.0467952         0   0.0460822   
..           ...      

In [14]:
from pyecharts.charts import Map
from pyecharts import options as opts
import pandas as pd
import numpy as np
def map_wuhan(data,Cmax):
    c = (
        Map()
        .add("武汉-Map", data, "武汉")
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Map-武汉地图"),
            visualmap_opts=opts.VisualMapOpts(max_=Cmax),
        )
    )
    return c
def map_beijing(data,Cmax):
    c = (
        Map()
        .add("北京-Map", data, "北京")
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Map-北京地图"),
            visualmap_opts=opts.VisualMapOpts(max_=Cmax),
        )
    )
    return c 

Wuhanqu=['江岸区','江汉区','硚口区','汉阳区','武昌区','青山区',
         '洪山区','蔡甸区','江夏区','黄陂区','新洲区','东西湖区','汉南区']
Beijingqu=['密云区','延庆区','朝阳区','丰台区','石景山区','海淀区','门头沟区', 
           '房山区','通州区','顺义区','昌平区','大兴区','怀柔区','平谷区','东城区','西城区']
values3= np.array([4300, 7290, 7239, 3579, 8224, 2958,4999,2588,1974,1661,1775,1018,1517])

values_renkou=np.array([96.24,72.96,86.85,65.27,127.63,52.88,163.75,73.50,91.37,98.83,90.21,56.25,13.55])
print(len(values_renkou))
values_severity=values3/values_renkou
values2=[8,9,20,3, 5, 7, 8, 2, 4,10,11,25,26,40,1,5]

data_H = []
for index in range(len(Wuhanqu)):
    city_info = [Wuhanqu[index], values3[index]]
    data_H.append(city_info)

data_wuhanrenkou = []
for index in range(len(Wuhanqu)):
    city_info = [Wuhanqu[index], values_renkou[index]]
    data_wuhanrenkou.append(city_info)
data_sevirity = []
for index in range(len(Wuhanqu)):
    city_info = [Wuhanqu[index], values_severity[index]]
    data_sevirity.append(city_info)
    
data_B = []
for index in range(len(Beijingqu)):
    city_info = [Beijingqu[index], values2[index]]
    data_B.append(city_info)

print(type(data_B))


a = map_wuhan(data_wuhanrenkou,max(values_renkou))
b = map_beijing(data_B,50)
c =  map_wuhan(data_sevirity,max(values_severity))
a.render(path="./mapresults/武汉人口地图.html")
b.render(path="./mapresults/beijing地图.html")
c.render(path="./mapresults/武汉疫情严重程度地图.html")
print(help(Map))

13
<class 'list'>
Help on class Map in module pyecharts.charts.basic_charts.map:

class Map(pyecharts.charts.chart.Chart, MapMixin)
 |  Map(init_opts: Union[pyecharts.options.global_options.InitOpts, dict] = <pyecharts.options.global_options.InitOpts object at 0x00000219185FE348>)
 |  
 |  `Base` is the root class for all graphical class, it provides
 |  part of the initialization parameters and common methods
 |  
 |  Method resolution order:
 |      Map
 |      pyecharts.charts.chart.Chart
 |      pyecharts.charts.base.Base
 |      pyecharts.charts.mixins.ChartMixin
 |      MapMixin
 |      builtins.object
 |  
 |  Methods inherited from pyecharts.charts.chart.Chart:
 |  
 |  __init__(self, init_opts: Union[pyecharts.options.global_options.InitOpts, dict] = <pyecharts.options.global_options.InitOpts object at 0x00000219185FE348>)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  set_colors(self, colors: Sequence[str])
 |  
 |  set_global_opts(self, title

In [32]:
Citys=['Beijing','Wuhan']
Wuhanqu=['江岸区','江汉区','硚口区','汉阳区','武昌区','青山区',
         '洪山区','蔡甸区','江夏区','黄陂区','新洲区','东西湖区','汉南区']
Beijingqu=['密云区','延庆区','朝阳区','丰台区','石景山区','海淀区','门头沟区', 
           '房山区','通州区','顺义区','昌平区','大兴区','怀柔区','平谷区','东城区','西城区']
A={'Beijing':Beijingqu,'Wuhan':Wuhanqu}
print(len(A['Beijing']))

16


In [19]:
print('Beijing' in 'Beijings')

True


In [37]:
Patches={'Beijing':Beijingqu,'Wuhan':Wuhanqu}
Wuhanqu=['江岸区','江汉区','硚口区','汉阳区','武昌区','青山区',
         '洪山区','蔡甸区','江夏区','黄陂区','新洲区','东西湖区','汉南区']
Beijingqu=['密云区','延庆区','朝阳区','丰台区','石景山区','海淀区','门头沟区', 
           '房山区','通州区','顺义区','昌平区','大兴区','怀柔区','平谷区','东城区','西城区']
print(len(Patches[i]) for i in ['Beijing','Wuhan'])

<generator object <genexpr> at 0x0000026C619DAC78>


In [55]:
a=1+-1
print(a)

0


In [18]:
print(216815/(13*(908.35+510.30)))
print(11.756301359092745*(908.35+510.30))

11.756301359092745
16678.076923076926


In [19]:
print(1>2 or 2>1)

True
